In [40]:
import os
import asyncio
from openai import AsyncAzureOpenAI
from openai import OpenAIError
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_tracing_disabled, function_tool

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

client = AsyncAzureOpenAI(
            api_version="2024-08-01-preview",
            azure_endpoint="https://aihub6750316290.openai.azure.com/",
            azure_ad_token_provider=token_provider,  # Use DefaultAzureCredential for authentication

        )
deployment = "gpt-4o"
model = OpenAIChatCompletionsModel(
                model=deployment,
                openai_client=client,
            )

In [41]:
from typing import List
from pydantic import BaseModel
from autogen_core.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
    FunctionExecutionResult
)
from typing import List, Literal, Optional, Union

class GroupChatMessage(BaseModel):
    body: LLMMessage
    conversation_id: str

class TransactionInfo(BaseModel):
    body: List[LLMMessage]
    conversation_id: str
    class Config:
        @staticmethod
        def schema_extra(schema: dict, model_class: type) -> None:
            # Remove defaults
            if "properties" in schema:
                for prop in schema["properties"].values():
                    prop.pop("default", None)
    

class AccountInfo(BaseModel):
    body: List[LLMMessage]
    conversation_id: str
    class Config:
        @staticmethod
        def schema_extra(schema: dict, model_class: type) -> None:
            # Remove defaults
            if "properties" in schema:
                for prop in schema["properties"].values():
                    prop.pop("default", None)


class NeedMoreInfo(BaseModel):
    body: List[LLMMessage]
    conversation_id: str



class IntermediateResult(BaseModel):
    body: List[LLMMessage]
    conversation_id: str 

TransactionAccountInfo = Union[TransactionInfo, AccountInfo, NeedMoreInfo, IntermediateResult]

FinalResponderAgentMessage = Union[IntermediateResult, NeedMoreInfo]

class GroupChatMessages(BaseModel):
    body: List[LLMMessage]
    conversation_id: str

In [42]:
sample_data_path = os.path.join(".", "sample_data")
@function_tool
def get_account_info(account_id: str) -> str:
    """Get account information for the given account ID."""
    try:
        account_data_file = os.path.join(sample_data_path, f"{account_id}.json")
        with open(account_data_file, "r") as file:
            data = file.read()
        return data
    except FileNotFoundError:
        return "Account not found."

@function_tool
def get_transaction_details(account_id: str) -> str:
    """Get transaction details for the given account ID."""
    try:
        txn_data_file = os.path.join(sample_data_path, f"Txn_{account_id}.json")
        with open(txn_data_file, "r") as file:
            data = file.read()
        return data
    except FileNotFoundError:
        return "Transaction details not found."

In [43]:
from agents import Agent, handoff, RunContextWrapper

get_account_info_agent = Agent(name="GetAccountInfoAgent",
                                  instructions="""You are Transify Support Agent. Transify is an online payment platform.
                                  You can provide information about the account details. 
                                  You need to know the 'account number' to provide the account details.
                                  If the 'account number' is provided use the get_account_info tool to get the account details.
                                  Format the response with 'Account Details:'.
                                  If the 'account number' is not provided, ask the user to provide the 'account number'. 
                                  If the account number is already provided, don't ask for it again.
                                  Summarize and format your response in a clear and concise manner.  
                                  Do not provide unnecessary information.

                                  Response Format as table:
                                    Account Balance: date | Balance Amount | account id
                                  """,
                                  tools=[get_account_info],
                                  model=model,
                                  #output_type=AccountInfo
                                  )


get_transaction_info_agent = Agent(name="GetTransactionDetailsAgent",
                                  instructions="""You are Transify Support Agent. Transify is an online payment platform.
                                  You can provide information about the transaction details for the account.
                                  You need to know the 'account number' to provide the transaction details.
                                  If the 'account number' is provided use the get_transaction_details tool to get the transaction details.
                                  Format the response with 'Transaction Details:'.
                                  If the 'account number' is not provided, ask the user to provide the 'account number'. 
                                  If the account number is already provided, don't ask for it again. 
                                  Summarize and format your response in a clear and concise manner.
                                  Do not provide eunecessary information. 

                                  Response Format as table:
                                    Transaction: date | amount | description | transaction id
                                  """,
                                  tools=[get_transaction_details],
                                  model=model,
                                  #output_type=TransactionInfo
                                  )

In [44]:
triage_agent = Agent(
    name="Triage agent",
    instructions=(
         """
        You are a Transify Support Agent Manager. Transify is an online payment platform.
        You can help users with their Transify related queries. 
        You need to respond only to queries related to Transify and nothing else. 
        If the question is not related to Transify, state that you only respond to Transify related queries and cannot answer this question.        
        If this is Transify related question. you need to find the appropriate agent based on the question as below:
        Agent Types:
        
        1. AccountInfo: If the question is about account details.
        2. TransactionInfo: If the question is about transaction details.
        """
    ),
    model=model,
    handoffs=[get_account_info_agent, get_transaction_info_agent],
)

In [62]:
result = await Runner.run(triage_agent, "I want to know my latest transaction details. ")
print(result.final_output)

Could you please provide your account number so I can retrieve your latest transaction details?


In [63]:
user_follow_input = result.to_input_list() + [{"role": "user", "content": "My account number is 1255."}]
result = await Runner.run(triage_agent, user_follow_input)
print(result.final_output)

It seems no transaction details are found for account number **1255**. Please ensure the account number is correct or let me know if you need further assistance.


In [64]:
user_follow_input = result.to_input_list() + [{"role": "user", "content": "My account number is A1234567890."}]
result = await Runner.run(triage_agent, user_follow_input)
print(result.final_output)

### Transaction Details:

| Transaction       | Date                | Amount   | Description                                | Transaction ID        |
|-------------------|---------------------|----------|--------------------------------------------|-----------------------|
| Latest Transaction| 29th September 2024 | $75.00   | Watercolor Paint Set & Brush Set          | 9HX25435AB0123456     |

Let me know if you need further details!


In [60]:
result = await Runner.run(triage_agent, "I want to know my latest transaction details. My account number is A1234567890.")
print(result.final_output)

**Transaction Details:**

| Transaction              | Date                 | Amount  | Description          | Transaction ID      |
|--------------------------|----------------------|---------|----------------------|---------------------|
| Latest Transaction       | 2024-09-29          | $75.00  | Watercolor Paint Set + Brush Set | 9HX25435AB0123456 |




In [51]:
result = await Runner.run(triage_agent, "My account number is A1234567890.")
print(result.final_output)

**Account Details:**

| **Account Balance:**       | Date                | Balance Amount | Account ID     |
|-----------------------------|---------------------|----------------|----------------|
|                            | 2024-10-09T10:20:15 | $150.00        | A1234567890    |


In [61]:
result = await Runner.run(triage_agent, "Plan a 3 day road trip to zion national park.")
print(result.final_output)

I only respond to Transify related queries and cannot answer this question. If you have any inquiries related to Transify, feel free to ask!
